# Experiment Tracking

使用wandb Python库用几行代码跟踪机器学习实验。如果您使用的是PyTorch或Keras等流行的框架，我们有轻量级集成。

然后，您可以在  interactive dashboard 中查看结果，或使用我们的公共API将数据导出到Python进行编程访问。

## Integrating W&B in your script

以下是跟踪 W&B 实验的简单构建模块。我们还为PyTorch、Keras、Scikit等提供了一系列特殊集成。请参阅我们的集成指南。

- `wandb.init()`: 在脚本顶部初始化新的运行。这返回一个 `Run` 对象并创建一个本地目录，其中保存所有日志和文件，然后异步流式传输到 W&B 服务器。如果您想使用私有服务器而不是我们的托管云服务器，我们提供自助托管。 
- `wandb.config`: 保存超参数的词典，如学习率或模型类型。您在配置中捕获的模型设置稍后可用于组织和查询结果。
- `wandb.log()`: 在训练循环中记录一段时间内的指标，例如准确性和损失。默认情况下，当您调用`wandb.log` 时，它会将新 step 附加到 `history` 对象并更新 `summary` 对象。
- - `history`: 一系列类似字典的对象，随着时间的推移跟踪指标。这些时间序列值在UI中显示为默认行图。
- - `summary`: 默认情况下，使用 `wandb.log()` 记录的指标的最终值。您可以手动设置指标的摘要，以捕获最高精度或最低损失，而不是最终值。这些值用于表中，以及比较 runs 的绘图——例如，您可以最终准确地可视化项目中的所有 runs 。
- `wandb.log_artifact`: 保存运行的输出，如模型权重或 a table of predictions。这不仅可以让您跟踪模型训练，还可以跟踪影响最终模型的所有 pipeline steps。

## Best Practices

`wandb` 非常灵活。以下是一些建议的指导方针。

- `Config`: 跟踪超参数、结构、数据集以及您想要用于重现模型的任何其他内容。这些将显示在列中 —— 使用配置列对 app 中的动态运行进行分组、排序和筛选。
- `Project`: project 是一组你可以比较的实验。每个 project 都有一个专用的 Dashboard 页面，您可以轻松打开和关闭不同的 runs 组，以比较不同的模型版本。
- `Notes`: 快速提交消息给自己， note 可以从脚本中设置，并在 table 中编辑。
- `Tags`: 识别baseline runs 和 最喜欢的 runs。您可以使用 tags 过滤运行，它们可以在 table 中编辑。

In [3]:
import wandb

config = dict(
    learning_rate = 0.01,
    momentum = 0.2, 
    architecture = "CNN",
    dataset_id = "peds-0192",
    infra = "AWS",
)

wandb.init(
    project="detect-pedestrains",
    notes="tweak baseline",
    tags=["baseline", "paper1"],
    config=config
)

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /Users/shawnd/.netrc


## What data is logged?

从脚本中记录的所有数据都保存在本地名为 `wandb` 的目录中，然后同步到 W&B 云或私有服务器。

### Logged Automatically

- **System metrics**: CPU 和 GPU利用率、网络等。这些显示在运行页面上的系统选项卡中。对于GPU，这些是用 nvidia-smi 获取的。
- **Command line**: stdout 和 stderr 被拾起并显示在运行页面上的日志选项卡中。

在帐户的设置页面中打开 Code Saving 以获取：

- **Git commit**: 拿起最新的 git 提交，并在运行页面的概览选项卡上查看，如果有任何未提交的更改，也可以查看 `diff.patch` 文件。
- **Dependencies**: `requirements.txt` 文件将上传并显示在运行页面的文件选项卡上，以及您保存到`wandb` 目录进行运行的任何文件。

### Logged with Specific Calls

就数据和模型指标而言，您可以确切地决定要记录的内容。

- **Dataset**: 您必须专门记录图像或其他数据集样本，以便它们流式传输到 W&B。
- **PyTorch gradients**: 添加 `wandb.watch(model)` 以在用户界面中将权重的梯度可视化为直方图。
- **Configuration info**: 记录超参数、数据集链接或用作配置参数的结构名称，像这样传递：`wandb.init(config=your_config_dictionary)`
- **Metrics**: 使用 `wandb.log` 查看模型中的指标。如果您在训练循环中记录准确性和损失等指标，您将在用户界面中获得实时更新图表。